In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# 中空層の熱抵抗を計算するモジュール

## 1. Functions  

面材間の表面温度から中空層の熱抵抗を計算する。

### 入力値  
$ \theta_1 $ ：面1の表面温度 (℃)  
$ \theta_2 $ ：面2の表面温度 (℃)  
$ \epsilon_1 $ ：面1の修正放射率  
$ \epsilon_2 $ ：面2の修正放射率  
$ s $ ：中空層の厚さ (m)  
gap_type_1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gap_type_2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gap_ratio_1：気体1の容積割合（0.0～1.0）  
gap_ratio_2：気体2の容積割合（1.0 - gas_ratio_1）  
gap_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）    

### 出力値
$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ \theta_1 $・$ \theta_2 $・$ \epsilon_1 $・$ \epsilon_2 $・$ s $・gap_type_1・gap_type_2・gap_ratio_1・gap_dirの組み合わせを1つの変数（タプル）で表し"gap_input"とする。  

In [2]:
class gap_input(namedtuple('gap_input','gap_Tmc1 gap_Tmc2 gap_eps1 gap_eps2 gap_s gap_type1 gap_type2 gap_ratio1 gap_dir')):
    __slots__ = ()

# gap_Tmc1：面1の表面温度（℃）
# gap_Tmc2：面2の表面温度（℃）
# gap_eps1：面1の修正放射率
# gap_eps2：面2の修正放射率
# gap_s：中空層の厚さ（m）
# gap_type1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gap_type2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gap_ratio1：気体1の容積割合（0.0～1.0）
# gap_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）

#### 中空層の熱抵抗の計算  

（参考：JIS R 3107：1998 4.1 基礎式）  

中空層の熱抵抗の計算方法を以下に示す。 

$ \begin{align}R_{s} = \frac{1}{h_r+h_g}\end{align}$ ････････････････････････････････････････ (5.1.3.2-a)  

ここで、  

$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ h_{r} $ ：中空層における放射熱コンダクタンス [W/(m<sup>2</sup>･K)]  
$ h_{g} $ ：中空層における気体の伝導と対流による熱コンダクタンス（気体熱コンダクタンス）[W/(m<sup>2</sup>･K)]  

である。 

#### 中空層の放射熱コンダクタンスの計算  

（参考：JIS R 3107：1998 4.2 放射熱コンダクタンス）  

中空層における放射熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_r = 4･\sigma･(\frac{1}{\epsilon_1}+\frac{1}{\epsilon_2}-1)^{-1}･T_m^3\end{align}$ ･･･････････････････････ (5.1.3.2-b)  


ここで、  

$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ \epsilon_1 $，$ \epsilon_2 $ ：中空層に接する二つのガラス面の修正放射率  
$ T_m $ ：中空層に接する二つのガラス面の絶対温度の平均値 (K)  

である。  

#### 中空層の気体熱コンダクタンスの計算  

（参考：JIS R 3107：1998 4.3 気体熱コンダクタンス）  

中空層における気体熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_g = Nu･\frac{\lambda}{s}\end{align}$ ･･････････････････････････････････････････ (5.1.3.2-c)

$ \begin{align}Nu = A･(Gr･Pr)^n\end{align}$ ････････････････････････････････････ (5.1.3.2-d)

$ \begin{align}Gr = \frac{9.81･s^3･\Delta T･\rho^2}{T'_m･\mu^2}\end{align}$ ････････････････････････････････ (5.1.3.2-e)

$ \begin{align}Pr = \frac{\mu･c}{\lambda}\end{align}$ ････････････････････････････････････････････ (5.1.3.2-f)  

ここで、  

$ Nu $ ：ヌセルト数  
$ \lambda $ ：気体の熱伝導率 [W/(m･K)]  
$ s $ ：中空層の厚さ (m)  
$ A $，$ n $ ：中空層の傾斜角と熱流方向によって決まる数値  
$ Gr $ ：グラスホフ数  
$ Pr $ ：プラントル数  
$ \Delta T $ ：中空層に接する二つのガラス面の温度差 (K)  
$ \rho $ ：気体の密度 (kg/m<sup>3</sup>)  
$ T'_m $ ：中空層の気体の平均絶対温度 (K)  
$ \mu $ ：気体の粘度 [kg/(m･s)]  
$ c $ ：気体の比熱 [J/(kg･K)]  

である。

中空層の傾斜角と熱流方向によって決まる数値$ A $，$ n $は、以下の値とする（参考：JIS R3107：1998 5.3.1）。

| 条件 | $A$ | $n$ |
| :--- | ---: | ---: |
| 中空層が垂直で熱流方向が水平の場合 | 0.035 | 0.38 |
| 中空層が水平で熱流方向が上向きの場合 | 0.16 | 0.28 |
| 中空層が45°で熱流方向が上向きの場合 | 0.10 | 0.31 |

また、混合気体の場合には、気体の物性値は、式(5.1.3-g)によって計算する（参考：JIS R3107：1998 5.3.2）。

$ \begin{align}F = \sum_{i=1}^N F_{i}･r_{i}\end{align} $ ･････････････････････････････････････････ (5.1.3.2-g)  

ここで、  

$ F $ ：混合気体の密度、粘度、熱伝導率又は比熱  
$ F_{i} $ ：各気体の密度、粘度、熱伝導率又は比熱  
$ r_{i} $ ：各気体の容積割合  
$ N $ ：気体の数  

である。  
なお、本仕様書では、$N$は1または2とする。

気体の物性値を以下に示す（参考：JIS R 3107：1998 付表2）。  

| 気体 | 温度 $\theta$ (℃) | 密度 $\rho$ (kg/m<sup>3</sup>) | 粘度 $\mu$ [kg/(m･s)] | 熱伝導率 $\lambda$ [W/(m･K)] | 比熱 $c$ [J/(kg･K)] |
| :--- | ---: | ---: | ---: | ---: | ---: |
| 空気 | -10.0 | 1.326 | 1.661e-5 | 2.336e-2 | 1.008e+3 |
|      | 0.0   | 1.277 | 1.711e-5 | 2.416e-2 | 1.008e+3 |
|      | 10.0  | 1.232 | 1.761e-5 | 2.496e-2 | 1.008e+3 |
|      | 20.0  | 1.189 | 1.811e-5 | 2.576e-2 | 1.008e+3 |
| アルゴン | -10.0 | 1.829 | 2.038e-5 | 1.584e-2 | 0.519e+3 |
|          | 0.0   | 1.762 | 2.101e-5 | 1.634e-2 | 0.519e+3 |
|          | 10.0  | 1.699 | 2.164e-5 | 1.684e-2 | 0.519e+3 |
|          | 20.0  | 1.640 | 2.228e-5 | 1.734e-2 | 0.519e+3 |
| SF<sub>6</sub> | -10.0 | 6.844 | 1.383e-5 | 1.119e-2 | 0.614e+3 |
|                | 0.0   | 6.602 | 1.421e-5 | 1.197e-2 | 0.614e+3 |
|                | 10.0  | 6.360 | 1.459e-5 | 1.275e-2 | 0.614e+3 |
|                | 20.0  | 6.118 | 1.497e-5 | 1.354e-2 | 0.614e+3 |
| クリプトン | -10.0 | 3.832 | 2.260e-5 | 0.842e-2 | 0.245e+3 |
|            | 0.0   | 3.690 | 2.330e-5 | 0.870e-2 | 0.245e+3 |
|            | 10.0  | 3.560 | 2.400e-5 | 0.900e-2 | 0.245e+3 |
|            | 20.0  | 3.430 | 2.470e-5 | 0.926e-2 | 0.245e+3 |

In [3]:
def Gap_Resistance(L):
    gap_ratio2 = 1. - L.gap_ratio1
    gap_dT = abs(L.gap_Tmc1 - L.gap_Tmc2)
    gap_Tmc = (L.gap_Tmc1 + L.gap_Tmc2) / 2.
    gap_Tm = gap_Tmc + 273.15
    ave_Tm = gap_Tm
    
    if L.gap_dir == 0:
        gap_A = 0.035
        gap_n = 0.38
    elif L.gap_dir == 1:
        gap_A = 0.16
        gap_n = 0.28
    elif L.gap_dir == 2:
        gap_A = 0.1
        gap_n = 0.31
    
    sgm = 5.67 * 10. ** (-8.)
    
    for j in range(1, 5):
        gas_mix = gas_prop(L.gap_type1, gap_Tmc, j) * L.gap_ratio1 + gas_prop(L.gap_type2, gap_Tmc, j) * gap_ratio2
        if j == 1: gas_rho = gas_mix
        elif j == 2: gas_mu = gas_mix * 10. ** (-5.)
        elif j == 3: gas_lmd = gas_mix * 10. ** (-2.)
        elif j == 4: gas_c = gas_mix * 10. ** 3.
    gas_Gr = 9.81 * L.gap_s ** 3. * gap_dT * gas_rho ** 2. / (gap_Tm * gas_mu ** 2.)
    gas_Pr = gas_mu * gas_c / gas_lmd
    gas_Nu = gap_A * (gas_Gr * gas_Pr)** gap_n
    if gas_Nu <= 1.: gas_Nu = 1.
    gap_hg = gas_Nu * gas_lmd / L.gap_s
    gap_hr = 4. * sgm * (1. / L.gap_eps1 + 1. / L.gap_eps2 - 1.) ** (-1.) * ave_Tm ** 3.
    return 1. / (gap_hr + gap_hg)

def gas_prop(gap_type, gap_Tmc, j):
    if gap_type == 0: #空気
        gp = np.array([[-10.0, 1.326, 1.661, 2.336, 1.008], [0.0, 1.277, 1.711, 2.416, 1.008],
                       [10.0, 1.232, 1.761, 2.496, 1.008], [20.0, 1.189, 1.811, 2.576, 1.008]])
    
    elif gap_type == 1: #アルゴン
        gp = np.array([[-10.0, 1.829, 2.038, 1.584, 0.519], [0.0, 1.762, 2.101, 1.634, 0.519],
                       [10.0, 1.699, 2.164, 1.684, 0.519], [20.0, 1.640, 2.228, 1.734, 0.519]])

    elif gap_type == 2: #SF6
        gp = np.array([[-10.0, 6.844, 1.383, 1.119, 0.614], [0.0, 6.602, 1.421, 1.197, 0.614],
                       [10.0, 6.360, 1.459, 1.275, 0.614], [20.0, 6.118, 1.497, 1.354, 0.614]])        
 
    elif gap_type == 3: #クリプトン
        gp = np.array([[-10.0, 3.832, 2.260, 0.842, 0.245], [0.0, 3.690, 2.330, 0.870, 0.245],
                       [10.0, 3.560, 2.400, 0.900, 0.245], [20.0, 3.430, 2.470, 0.926, 0.245]])         
    
    for i in range(1, 4):
        if i == 1 and gap_Tmc < gp[i-1, 0]: break
        elif gp[i-1, 0] <= gap_Tmc and gap_Tmc < gp[i, 0]: break
        elif i == 3 and gp[i, 0] <= gap_Tmc: break
       
    gp_cal = gp[i - 1, j] + (gp[i, j] - gp[i - 1, j]) * (gap_Tmc - gp[i - 1, 0]) / (gp[i, 0] - gp[i - 1, 0])
    
    return gp_cal

#### Example

In [4]:
if __name__ == '__main__':

    Rgap_in = [gap_input(0., 20., 0.837, 0.837, 0.012, 0, 0, 1., 0),
               gap_input(30., 25., 0.837, 0.837, 0.012, 0, 0, 1., 0),
               gap_input(0., 20., 0.837, 0.837, 0.012, 1, 0, 0.8, 0)]

    Rgap_result = np.zeros(len(Rgap_in))
    for i in range(len(Rgap_in)):
        Rgap_result[i] = Gap_Resistance(Rgap_in[i])

    print (Rgap_result)

[0.17284803 0.15077336 0.19069065]
